# 0-Tokenizer

对一段文本进行分词就是将其分割成单词或子词，然后通过查找表将这些单词或子词转换为对应的编号。

分词器的就是把一段文本分割成单词或子词的工具。 

## 子词分词算法

常见的子词分词算法有三种：

1. 字节对编码（Byte Pair Encoding，BPE）
2. WordPiece
3. Unigram

### BPE

BPE 是一种简单的数据压缩技术，它会迭代地替换序列中最频繁出现的字节对。BPE 依赖一个预分词器，该预分词器会将训练数据分割成单词（在本项目中，我们使用按空格分词的方法作为预分词方法）。

在预分词之后，会创建一组唯一的单词，并确定它们在数据中的出现频率。接下来，BPE 会创建一个基础词表，该词表包含预分词器最初生成的数据中所有唯一单词的符号。然后，会将这对符号从词表中移除，新形成的符号将加入词表。在迭代过程中，BPE 算法会合并频繁出现的符号对。

给定词表的大小，BPE（字节对编码）算法最终会合并出现频率最高的符号对，直到收敛到该大小。

### WordPiece

WordPiece 算法与 BPE 非常相似。WordPiece 首先将词表初始化为包含训练数据中出现的每个字符，然后逐步学习给定数量的合并规则。与 BPE 不同的是，WordPiece 并不选择最频繁出现的符号对，而是选择那个加入词表后能使训练数据出现的可能性最大化的符号对。

### Unigram

Unigram 算法将其基础词表初始化为大量的符号，然后逐步削减每个符号，以获得一个更小的词表。它会在训练数据上定义一个对数似然损失，以此来确定是否从词表中移除某个符号。

## 训练一个最简单的分词器

在本节中，我们将学习基于 transformers 库来训练你自己的分词器。

### 初始化

首先，我们应该初始化我们的分词器，并确定选择哪种方法。我们将使用字节对编码（BPE）算法。

In [1]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

### 定义特殊标记

数据集中存在一些我们不希望被分词的特殊标记，我们会将这些标记定义为特殊标记，并将它们传递给分词器训练器，以防止出现错误的分词情况。 

In [2]:
special_tokens = ["<unk>", "<s>", "</s>"]

trainer = trainers.BpeTrainer(
        vocab_size=256,
        special_tokens=special_tokens,
        show_progress=True,
        initial_alphabet=pre_tokenizers.ByteLevel.alphabet()
)

### 从文件中读取数据
我们使用 JSON Lines（jsonl）格式来存储训练数据，让我们编写函数来处理这些数据。

In [3]:
import json

def read_texts_from_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            yield data['text']

data_path = './demo/tokenizer_data.jsonl'

In [4]:
data_iter = read_texts_from_jsonl(data_path)
next(data_iter)

'<s>近年来，人工智能技术迅速发展，深刻改变了各行各业的面貌。机器学习、自然语言处理、计算机视觉等领域的突破性进展，使得智能产品和服务越来越普及。从智能家居到自动驾驶，再到智能医疗，AI的应用场景正在快速拓展。随着技术的不断进步，未来的人工智能将更加智能、更加贴近人类生活。</s>'

### 开始训练!

我们使用分词器的内置函数来训练分词器，轻松愉快。

In [5]:
tokenizer.train_from_iterator(data_iter, trainer=trainer)

### 设置解码器

In [6]:
tokenizer.decoder = decoders.ByteLevel()

接下来，检查一下特殊标记是否得到了妥善处理。

In [7]:
assert tokenizer.token_to_id('<unk>') == 0
assert tokenizer.token_to_id('<s>') == 1
assert tokenizer.token_to_id('</s>') == 2

### 将训练好的分词器保存到磁盘

In [8]:
import os

tokenizer_dir = "./demo/tokenizer_demo"
os.makedirs(tokenizer_dir, exist_ok=True)
tokenizer.save(os.path.join(tokenizer_dir, "tokenizer.json")) # At this point, you will see a file named tokenizer.json under tokenizer_dir
tokenizer.model.save(tokenizer_dir) # generate vocab.json & merges.txt

['./demo/tokenizer_demo\\vocab.json', './demo/tokenizer_demo\\merges.txt']

### 手动创建一份配置文件

In [9]:
config = {
    "add_bos_token": False,
    "add_eos_token": False,
    "add_prefix_space": False,
    "added_tokens_decoder": {
        "0": {
            "content": "<unk>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
        },
        "1": {
            "content": "<s>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
        },
        "2": {
            "content": "</s>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
        }
    },
    "additional_special_tokens": [],
    "bos_token": "<s>",
    "clean_up_tokenization_spaces": False,
    "eos_token": "</s>",
    "legacy": True,
    "model_max_length": 32768,
    "pad_token": "<unk>",
    "sp_model_kwargs": {},
    "spaces_between_special_tokens": False,
    "tokenizer_class": "PreTrainedTokenizerFast",
    "unk_token": "<unk>",
    "chat_template": "{{ '<s>' + messages[0]['text'] + '</s>' }}"
}

with open(os.path.join(tokenizer_dir, "tokenizer_config.json"), "w", encoding="utf-8") as config_file:
    json.dump(config, config_file, ensure_ascii=False, indent=4)

print("Tokenizer training completed and saved.")

Tokenizer training completed and saved.


现在我们已经训练了一个简单的分词器，并将其进行保存，接下来，我们试着加载它，并使用其帮助我们对文本进行编解码

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./demo/tokenizer_demo")
msg = [{"text": "失去的东西就要学着去接受，学着放下。"}]
new_msg = tokenizer.apply_chat_template(
    msg,
    tokenize=False
)
new_msg

'<s>失去的东西就要学着去接受，学着放下。</s>'

In [11]:
actual_vocab_size = len(tokenizer)
print('actural length of tokenizer：', actual_vocab_size)

actural length of tokenizer： 259


In [12]:
model_inputs = tokenizer(new_msg)
model_inputs

{'input_ids': [1, 164, 100, 112, 164, 239, 122, 166, 251, 229, 163, 119, 253, 167, 101, 126, 164, 111, 112, 167, 102, 226, 164, 258, 102, 166, 254, 225, 164, 239, 122, 165, 239, 101, 164, 240, 248, 174, 123, 237, 164, 258, 102, 166, 254, 225, 165, 245, 125, 163, 119, 236, 162, 225, 227, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
response = tokenizer.decode(model_inputs['input_ids'], skip_special_tokens=False)
response

'<s>失去的东西就要学着去接受，学着放下。</s>'

In [14]:
response = tokenizer.decode(model_inputs['input_ids'], skip_special_tokens=True)
response

'失去的东西就要学着去接受，学着放下。'